# São Paulo Beaches Water Quality

The objective of this project is to build a database of the measures of Enterococcus(UFC/100ml) in some beaches in São Paulo state. Lower numbers of Enterococcus indicate better water quality. The measures are done once a week in most of the stations (some are done monthly) since 2012.

In [ ]:
import re
import datetime
import tabula
import csv
import os

import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
def clean(value):
    """Clean some of the fieds in the tables. If it can't be trated in this function, keep the same value but
    print a warning so we know we have to fix it manually.
    """
    
    try:
        return int(value)
    except (ValueError, TypeError):
        v = str(value).strip()

        if '*' in v:
            return clean(v.replace('*', '.'))
        if ',' in v:
            return clean(v.replace(',', '.'))
        if '.' in v:
            return clean(v.replace('.', ''))
        if v == '':
            return 0
        if v == 'nan':
            return 0
        if v == '<1':
            return 1
        if re.match(r'[\d]+[\s][\d]+', v):
            return v.replace(' ', ',')
        print('> Clean err:', value, v)
    return value

In [ ]:
def name_clean(value):
    """Clean string values."""
    
    if '*' in value:
        value = value.replace('*', '')
    return value.strip()

In [ ]:
def date_header(days, year):
    """Build the header transforming the list of days into complete date. If the file can't be parsed properly
    it should keep only the days and print a warning so we know we have to fix it manually.
    """
    
    try: 
        month_header = []
        month = 1
        for i in range(len(days)):
            prev, cur = int(days[i - 1]), int(days[i])
            if i > 1 and cur - prev < 0 and cur != 0 and prev != 0:
                month += 1
            month_header.append(datetime.date(year, month, int(days[i])))
        return month_header
    except ValueError:
        print('> Header err')
        return days

In [ ]:
# List of files to be parsed. Each file have several tables that need to be parsed.
files = [
    [2020, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-baixada-santista_2020.pdf'],
    [2020, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-litoral-norte_2020.pdf'],
    [2019, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-baixada-santista_2019.pdf'],
    [2019, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-litoral-norte_2019.pdf'],
    [2018, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-baixada-santista_2018.pdf'],
    [2018, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-litoral-norte_2018.pdf'],
    [2017, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-baixada-santista_2017.pdf'],
    [2017, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-litoral-norte_2017.pdf'],
    [2016, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-baixada-santista_2016.pdf'],
    [2016, 'https://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-litoral-norte_2016.pdf'],
    [2015, 'http://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-baixada-santista_2015.pdf'],
    [2015, 'http://sistemasinter.cetesb.sp.gov.br/praias/excel/resultado-microbiologico-litoral-norte_2015.pdf'],
    [2014, 'https://cetesb.sp.gov.br/praias/wp-content/uploads/sites/31/2013/11/resultado-microbiologico-baixada-santista1.pdf'],
    [2014, 'https://cetesb.sp.gov.br/praias/wp-content/uploads/sites/31/2013/11/resultado-microbiologico-litoral-norte1.pdf'],
    [2013, 'https://cetesb.sp.gov.br/praias/wp-content/uploads/sites/31/2013/11/resultado-microbiologico-baixada-santista.pdf'],
    [2013, 'https://cetesb.sp.gov.br/praias/wp-content/uploads/sites/31/2013/11/resultado-microbiologico-litoral-norte.pdf'],
    [2012, 'https://cetesb.sp.gov.br/praias/wp-content/uploads/sites/31/2013/11/resultado-microbiologico-2012-baixada-santista.pdf'],
    [2012, 'https://cetesb.sp.gov.br/praias/wp-content/uploads/sites/31/2013/11/resultado-microbiologico-2012-litoral-norte.pdf']
]

In [ ]:
# Loop through the files, parse the tables in it, save in the folder data and print warnings when the file need
# to be fixed manually.
for year, filename in files:
        
    file = tabula.read_pdf(filename, pages='all')
    
    for data in file:
        city = name_clean(data.columns[0])
        print(year, city)
        name = f'data/{year}_{city}.csv'
        
        df = data.fillna(0)
        df = df.iloc[1:,1:].applymap(clean) 
        df.columns = date_header(list(data.iloc[0, 1:]), year)
        df.index = data.iloc[1:,0]
        df.to_csv(name)

In [ ]:
# Loop through the files with tables and do some more cleaning (quotes and spaces)
for filename in os.listdir('data/'):
    f = open(f'data/{filename}', 'r', encoding='utf8')
    content = f.read()
    f.close()
    
    f_new = open(f'processed/{filename}', 'w', encoding="utf8")
    f_new.write(content.replace('\"', '').replace(' ,', ','))    
    f_new.close()

I copied the files in the folder `processed` to `fixed` and manually fixed the files that had parsing error messages. This was necessary, since each file have a different table format and it was difficult to process all using one rule.

In [ ]:
final = pd.DataFrame(columns=['City', 'Beach', 'Date', 'Enterococcus'])

for filename in os.listdir('fixed/'):
    df = pd.read_csv(f'fixed/{filename}')
    
    city = df.columns[0]
    
    rows, cols = df.shape
    for r in range(rows):
        beach = df.iloc[r, 0]
        for c in range(1, cols):
            val = df.iloc[r, c]
            if val > 0:
                final = final.append({'City': city, 'Beach': beach, 'Date': df.columns[c], 'Enterococcus': val}, ignore_index=True)

final.to_csv('sp_beaches.csv', index=False)